# Working with sockets

In [2]:
import socket

# Working with blocking sockets

# Creating simple socket-server with host+port with protocol TSP
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# To be able use same port
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# Defining host+port of server
server_address = ("127.0.0.1", 8000)
server_socket.bind(server_address)

#listening incoming requests
server_socket.listen()
connection, client_address = server_socket.accept()
print(f"f'I got a connection from {client_address}!")

f'I got a connection from ('127.0.0.1', 51366)!


"\r\n" - end in telnet
We cannot get all data at one moment, so we use buffer and receive data by parts.

In [10]:
import socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

server_address = ('127.0.0.1', 8000)
server_socket.bind(server_address)
server_socket.listen()

try:
    connection, client_address = server_socket.accept()
    print(f'I got a connection from {client_address}!')

    buffer = b''

    # Receive data till we get end symbols
    while buffer[-2:] != b'\r\n':
        data = connection.recv(2)
        if not data:
            break
        else:
            print(f'I got data: {data}!')
            buffer = buffer + data
    
    print(f"All the data is: {buffer}")
    connection.sendall(buffer)
finally:
    server_socket.close()

I got a connection from ('127.0.0.1', 51585)!
I got data: b't'!
I got data: b'e'!
I got data: b's'!
I got data: b't'!
I got data: b'i'!
I got data: b'n'!
I got data: b'g'!
I got data: b'1'!
I got data: b'2'!
I got data: b'3'!
I got data: b'\r\n'!
All the data is: b'testing123\r\n'


Curreny this server above is not useful because it can work only with 1 user but we can make server be able to work with multiple users.

In [11]:
import socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_address = ('127.0.0.1', 8000)
server_socket.bind(server_address)
server_socket.listen()
connections = []

# This is not also not working because sockets are blocking, they will wait till next operation blocking all others
# We can do it bu this line below but then we need to catch connections because we will get exception BlockingIOerror 
"server_socket.setblocking(False)"

try:
    while True:
        connection, client_address = server_socket.accept()
        print(f'I got a connection from {client_address}!')
        "connection.setblocking(False)"
        connections.append(connection)
        for connection in connections:
            buffer = b''
            while buffer[-2:] != b'\r\n':
                data = connection.recv(2)
                if not data:
                    break
                else:
                    print(f'I got data: {data}!')
                    buffer = buffer + data
            
            print(f"All the data is: {buffer}")
            connection.send(buffer)
finally:
    server_socket.close()

I got a connection from ('127.0.0.1', 51611)!
I got data: b't'!
I got data: b'e'!
I got data: b's'!
I got data: b't'!
I got data: b'i'!
I got data: b'n'!
I got data: b'g'!
I got data: b'1'!
I got data: b'2'!
I got data: b'3'!
I got data: b'\r\n'!
All the data is: b'testing123\r\n'
I got a connection from ('127.0.0.1', 51614)!
I got data: b'an'!
I got data: b'ot'!
I got data: b'he'!
I got data: b'rt'!
I got data: b'es'!
I got data: b't1'!
I got data: b'23'!
I got data: b'\r\n'!
All the data is: b'anothertest123\r\n'
I got data: b'a'!
I got data: b'n'!
I got data: b'o'!
I got data: b't'!
I got data: b'h'!
I got data: b'e'!
I got data: b'r'!
I got data: b't'!
I got data: b'e'!
I got data: b's'!
I got data: b't'!
I got data: b'1'!
I got data: b'2'!
I got data: b'3'!
I got data: b'\r\n'!
All the data is: b'anothertest123\r\n'


We can work with non-blocking sockets with selectors so it will not heavy for our CPU 

In [3]:
import selectors
import socket
from selectors import SelectorKey
from typing import List, Tuple

# Created selector that will detect sockets, incoming calls
selector = selectors.DefaultSelector()

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_address = ('127.0.0.1', 8000)
server_socket.setblocking(False)
server_socket.bind(server_address)
server_socket.listen()

# Registering our server socket
selector.register(server_socket, selectors.EVENT_READ)

wait = 0
# We can see that this is kinda simillar how works event loop in asyncio
while True:
    events: List[Tuple[SelectorKey, int]] = selector.select(timeout=1)

    if wait == 10:
        break
    if len(events) == 0:
        wait += 1
        print("No events, waiting for new event")
    
    for event, _ in events:
        wait = 0
        event_socket = event.fileobj

        if event_socket == server_socket:
            connection, address = server_socket.accept()
            connection.setblocking(False)
            print(f"I got connection from {address}")
            selector.register(connection, selectors.EVENT_READ)
        else:
            data = event_socket.recv(1024)
            print(f"All the data is: {data}")
            connection.send(data)


No events, waiting for new event
No events, waiting for new event
No events, waiting for new event
No events, waiting for new event
No events, waiting for new event
No events, waiting for new event
No events, waiting for new event
No events, waiting for new event
No events, waiting for new event
No events, waiting for new event


Let`s make echo-server using asyncio without selectors

In [1]:
import asyncio
import socket

async def echo(connection: socket, loop : asyncio.AbstractEventLoop) -> None:
    buffer = b''

    # This code can work concurrently, provide connections with several users and close them by 'quit'
    while True:
        while buffer[-2:] != b'\r\n':
            data = await loop.sock_recv(connection, 1024)
            if not data:
                break
            else:
                buffer = buffer + data

        if buffer[:-2] == b'quit':
            # We can also raise exception but then we need to catch it by try/except
            # But if one person get exception, other users connected to server also will lose connection, we kinda lose concurrency
            break
        
        print(f"All the data is: {buffer[:-2]}")
        await loop.sock_sendall(connection, buffer)
        buffer = b''

async def listen_for_connections(server_socket : socket, loop: asyncio.AbstractEventLoop) -> None:
    while True:
        connection, address = await loop.sock_accept(server_socket)
        connection.setblocking(False)
        print(f"Got a connection from {address}")
        asyncio.create_task(echo(connection, loop))

async def main():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    server_address = ('127.0.0.1', 8000)
    server_socket.setblocking(False)
    server_socket.bind(server_address)
    server_socket.listen()

    await listen_for_connections(server_socket, asyncio.get_event_loop())


await main()

Got a connection from ('127.0.0.1', 54301)
All the data is: b'testing123'
All the data is: b'gg it works!'
